# "Hello World" Example in R

In [1]:
library(glmnet)
library(rjson)
library(tidyverse)
library(MASS)
library(purrr)

Loading required package: Matrix
Loaded glmnet 3.0

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select



## Researcher Loads in Data

In [5]:
#X = read.table('../X.csv', sep=',')
#y = read.table('../Y.csv')

## Selection Algorithm (function)

Relevant metadata:

```
"capture_selection": [
  {
    "selection_type": "set",
    "encoder": "dataframe",
    "name": "selected_vars"
  }
```

Using a map/dictionary metaphor, the "keys" are universal elements of the algorithm. The "values" are the corresponding names of these elements in the notebook code.

After the initial run of the preprocessor on the full dataset, we add an additional entry to the metadata:

```
"original_selection": "some base 64 string"
```

where `"some base 64 string"` is a base 64 string of the dataframe of selected variables

In [6]:
selection_algorithm <- function(X, y) {
    X = as.matrix(X)
    y = as.matrix(y)
    n <- nrow(X)
    p <- ncol(X)
    
    min_success <- 6
    ntries <- 10
    
    make_lambda_grid <- function(X, y) {
        # Return a vector of lambda values where the corresponding lasso model
        # satisfies the following constraint:
        #   number of selected variables < sqrt(0.8 * p)
        p <- ncol(X)
        model <- cv.glmnet(X, y, alpha=1)$glmnet.fit
        lambdas <- model$lambda
        nselected <- model$df  # number of selected vars for each lambda
        return(lambdas[nselected < sqrt(0.8 * p)])
    }

    lambda_grid <- make_lambda_grid(X, y)
    success <- matrix(0, nrow=p, ncol=length(lambda_grid))
    
    for(i in 1:ntries) {
        subsample_indexes <- sample(1:n, floor(n/2), replace=FALSE)
        Xsub <- X[subsample_indexes,]
        noisy_y <- y[subsample_indexes] + rnorm(floor(n/2))
        model <- cv.glmnet(Xsub, noisy_y, alpha=1)
        
        lambdas <- model$lambda
        coefs <- coef(model, lambda_grid)
        success <- success + (coefs[-1,] != 0)
    }
    
    selection_fn <- function(x) {
        return(sum(x > min_success) > 2)
    }
    selected <- apply(X=success, MARGIN=2, FUN=selection_fn)
    vars <- which(selected != 0)  # indexes of selected lambdas
    return(as.numeric(vars))
}

selected_vars <- selection_algorithm(X, y)
selected_vars <- data.frame(selection = selected_vars)
#selected_vars

## Sufficient Statistics, Estimators, Simulation

Relevant metadata:

```
"functions": "stats_computations",
"data_model": {
  "sufficient_statistics": "compute_sufficient_statistics",
  "estimators": "compute_estimators",
  "resample_data": "resample_data"
}
```

The "keys" are universal elements of the algorithm. The "values" are the corresponding names of these elements in the notebook code.

In [9]:
M = lm(as.matrix(y) ~ as.matrix(X))
D = model.matrix(M)
compute_sufficient_statistics <- function(selection) {
    # Computes the sufficient statistic and returns it as a dataframe
    y_ = as.matrix(y)   
    suff_stat_1 <- t(D) %*% y_
    suff_stat_2 <- sum(y^2)
    
    combined <- c(suff_stat_1) #
    combined  <- data.frame(t(as.matrix(combined)))
    
    return(combined)
}

test_fn = function(selection,idx) { idx %in% selection[['selected_vars']][['selection']]}

compute_estimators <- function(selection) {
    #X = as.matrix(data[["X"]])
    #y = as.matrix(data[["y"]])
    
    result = list()
    beta_hat = coef(M)[-1]
    idx = 1
    for (var in c(selection[['selected_vars']][['selection']])) {
        fn = purrr::partial(test_fn, idx=!!var) # the !!unquotes var so its value is fixed
        result[[idx]] = list(identifier=var, 
                             value=beta_hat[var], 
                             check_fn=fn)
        idx = idx + 1
    }
    return(result)
}

estimate_var <- function(selection) {
    V = diag(vcov(M))[-1]
    result = list() 
    idx = 1
    n = nrow(X)
    p = ncol(D)
    dispersion = sum(resid(M)^2) / (n-p)
    for (var in c(selection[['selected_vars']][['selection']])) {
        fn = purrr::partial(test_fn, idx=!!var) # the !!unquotes var so its value is fixed
        cross = rep(0, p)
        cross[var] = dispersion
        result[[idx]] = list(identifier=var, 
                             var=V[var],
                             cross=cross)
        idx = idx + 1
    }    
    return(result)
}

resample_data <- function(data, selection) {
    X = as.matrix(data[["X"]])
    y = as.matrix(data[["y"]])
    #fixed_sel <- fromJSON(fixed_sel)
    #n <- nrow(X)
    #p <- ncol(X)
    #resids <- y - X %*% ginv(t(X) %*% X) %*% (t(X) %*% y)
    #fitted <- y - resids
    
    #resampled <- sample(1:n, n, replace=TRUE)
    #y_tilde <- fitted + resids[resampled]
    y_tilde <- y + rnorm(n = dim(y)[1])
    
    return(list(X = X, y = y_tilde))
}

# Test/display resampling
#data = list(X = X, y = y)
#resample_data(data, selected_vars)

## Selection Indicators

Here, we define two choices of selection indicator functions -- one for fixed selection and one for set selection. The user can pick either, depending on the type of inference being performed.

Relevant metadata:

```
"functions": "sel_indicators",
"data_model": {
  "selection_indicator_function": "get_fixed_sel_indicators"
}```

# TODO: In preprocessor, inject a cell that saves the original selection in the
# kernel. This allows us to compare original_sel_vars with the simulated
# sel vars later.

get_fixed_sel_indicator <- function(original_selection, simulated_selection) {
    # Generates a single indicator variable (1 or 0) for the fixed selection
    # of the simulated data (compared to the original sample).
    
    fixed_sel_indicator <- all.equal(original_selection['selected_vars'], simulated_selection['selected_vars'])
    return(fixed_sel_indicator)  # single indicator
}

get_set_sel_indicators <- function(original_selection, simulated_selection) {
    # Generates a 1-D dataframe of selection indicators (1 or 0) for the set
    # selection of the simulated data (compared to the original sample).

    # This specific function assumes original_sel_vars is one-dimensional, but
    # this doesn't necessarily have to hold as long as the output selection
    # indicators are in a one-dimensional array.
    
    original_sel_vars = original_selection[['selected_vars']]
    simulated_sel_vars = simulated_selection[['selected_vars']]
    # Empty vector of selection indicators
    set_sel_indicators <- c()
    
    # Loop over each original selected variable to see if it is also selected
    # in the simulated data.
    sel_var_count = dim(original_sel_vars)[1]
    for(i in 1:sel_var_count) {
        sel_indicator <- original_sel_vars[i,1] %in% simulated_sel_vars[,1]
        set_sel_indicators <- c(set_sel_indicators, sel_indicator)
    }
    
    # Return the vector of indicators as a data frame
    set_sel_indicators <- data.frame(set_sel_indicators)
    return(set_sel_indicators)
}
